In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from datetime import datetime

import cv2
import numpy as np
from bytetracker import BYTETracker
from bytetracker.basetrack import BaseTrack
from tqdm import tqdm

from lib.bbox.utils import rescale_bbox, xy_center_to_xyxy
from lib.sequence import Sequence
from trackreid.args.reid_args import OUTPUT_POSITIONS
from trackreid.reid_processor import ReidProcessor

sys.path.append("..")


Baseline data can be found in `gs://data-track-reid/predictions/baseline`. You can copy them in `../data/predictions/` using the following commands (in a terminal at the root of the project):

```bash
mkdir -p ./data/predictions/
gsutil -m cp -r gs://data-track-reid/predictions/baseline ./data/predictions/
```

Then you can reoganize the data using the following : 
```bash 
find ./data/predictions/baseline -mindepth 2 -type f -name "*.txt" -exec sh -c 'mv "$0" "${0%/*/*}/$(basename "${0%/*}").txt"' {} \; && find ./data/predictions/baseline -mindepth 1 -type d -empty -delete
```

# Real life data

In [ ]:
DATA_PATH = "../data"
DETECTION_PATH = f"{DATA_PATH}/detections"
FRAME_PATH = f"{DATA_PATH}/frames"
PREDICTIONS_PATH = f"{DATA_PATH}/predictions"
VIDEO_OUTPUT_PATH = "private"

SEQUENCES = os.listdir(DETECTION_PATH)
GENERATE_VIDEOS = False


In [ ]:
def get_sequence_frames(sequence):
    frames = os.listdir(f"{FRAME_PATH}/{sequence}")
    frames = [os.path.join(f"{FRAME_PATH}/{sequence}", frame) for frame in frames]
    frames.sort()
    return frames

def get_sequence_detections(sequence):
    detections = os.listdir(f"{DETECTION_PATH}/{sequence}")
    detections = [os.path.join(f"{DETECTION_PATH}/{sequence}", detection) for detection in detections]
    detections.sort()
    return detections

In [ ]:
class DetectionHandler():
    def __init__(self, image_shape) -> None:
        self.image_shape = image_shape

    def process(self, detection_output):
        if detection_output.size:
            if detection_output.ndim == 1:
                detection_output = np.expand_dims(detection_output, 0)

            processed_detection = np.zeros(detection_output.shape)

            for idx, detection in enumerate(detection_output):
                clss = detection[0]
                conf = detection[5]
                bbox = detection[1:5]
                xyxy_bbox = xy_center_to_xyxy(bbox)
                rescaled_bbox = rescale_bbox(xyxy_bbox,self.image_shape)
                processed_detection[idx,:4] = rescaled_bbox
                processed_detection[idx,4] = conf
                processed_detection[idx,5] = clss

            return processed_detection
        else:
            return detection_output


In [ ]:
class TrackingHandler():
    def __init__(self, tracker) -> None:
        self.tracker = tracker

    def update(self, detection_outputs, frame_id):

        if not detection_outputs.size :
            return detection_outputs

        processed_detections = self._pre_process(detection_outputs)
        tracked_objects = self.tracker.update(processed_detections, frame_id = frame_id)
        processed_tracked = self._post_process(tracked_objects)
        return processed_tracked

    def _pre_process(self,detection_outputs : np.ndarray):
        return detection_outputs

    def _post_process(self, tracked_objects : np.ndarray):

        if tracked_objects.size :
            if tracked_objects.ndim == 1:
                tracked_objects = np.expand_dims(tracked_objects, 0)

        return tracked_objects

In [ ]:
def bounding_box_distance(obj1, obj2):
    # Get the bounding boxes from the Metadata of each TrackedObject
    bbox1 = obj1.metadata.bbox
    bbox2 = obj2.metadata.bbox

    # Calculate the Euclidean distance between the centers of the bounding boxes
    center1 = ((bbox1[0] + bbox1[2]) / 2, (bbox1[1] + bbox1[3]) / 2)
    center2 = ((bbox2[0] + bbox2[2]) / 2, (bbox2[1] + bbox2[3]) / 2)
    distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)

    return distance

# TODO : discard by zone
def select_by_category(obj1, obj2):
    # Compare the categories of the two objects
    return 1 if obj1.category == obj2.category else 0

In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
print(timestamp)
folder_save = os.path.join(PREDICTIONS_PATH,timestamp)
os.makedirs(folder_save, exist_ok=True)
GENERATE_VIDEOS = False
for sequence in tqdm(SEQUENCES) :
    frame_path = get_sequence_frames(sequence)
    test_sequence = Sequence(frame_path)
    frame_id = 0
    BaseTrack._count = 0
    from datetime import datetime

    file_path = os.path.join(folder_save,sequence) + '.txt'
    video_path = os.path.join(folder_save,sequence) + '.mp4'

    if GENERATE_VIDEOS:
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # or use 'x264'
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (2560, 1440))  # adjust the frame size (640, 480) as per your needs

    detection_handler = DetectionHandler(image_shape=[2560, 1440])
    tracking_handler = TrackingHandler(tracker=BYTETracker(track_thresh= 0.3, track_buffer = 5, match_thresh = 0.85, frame_rate= 30))
    reid_processor = ReidProcessor(filter_confidence_threshold=0.1,
                                    filter_time_threshold=5,
                                    cost_function=bounding_box_distance,
                                    cost_function_threshold=5000, # max cost to rematch 2 objects
                                    selection_function=select_by_category,
                                    max_attempt_to_match=5,
                                    max_frames_to_rematch=500,
                                    save_to_txt=True,
                                    file_path=file_path)

    for frame, detection in test_sequence:

        frame_id += 1

        processed_detections = detection_handler.process(detection)
        processed_tracked = tracking_handler.update(processed_detections, frame_id)
        reid_results = reid_processor.update(processed_tracked, frame_id)

        if GENERATE_VIDEOS and len(reid_results) > 0:
            frame = np.array(frame)
            for res in reid_results:
                object_id = int(res[OUTPUT_POSITIONS["object_id"]])
                bbox = list(map(int, res[OUTPUT_POSITIONS["bbox"]]))
                class_id = int(res[OUTPUT_POSITIONS["category"]])
                tracker_id = int(res[OUTPUT_POSITIONS["tracker_id"]])
                mean_confidence = float(res[OUTPUT_POSITIONS["mean_confidence"]])
                x1, y1, x2, y2 = bbox
                color = (0, 0, 255) if class_id  else (0, 255, 0)  # green for class 0, red for class 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{object_id} ({tracker_id}) : {round(mean_confidence,2)}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        if GENERATE_VIDEOS:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            out.write(frame)

    if GENERATE_VIDEOS :
        out.release()




In [ ]:
from collections import defaultdict

def count_occurrences(file_path, case):
    object_counts = defaultdict(int)
    class_counts = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r') as f:
        for line in f:
            data = line.split()

            if case != 'baseline':
                object_id = int(data[1])
                category = int(data[2])
            else:
                object_id = int(data[1])
                category = int(data[-1])

            object_counts[object_id] += 1
            class_counts[object_id][category] += 1

    return object_counts, class_counts

def filter_counts(object_counts, class_counts, min_occurrences=10):
    filtered_objects = {}

    for object_id, count in object_counts.items():
        if count > min_occurrences and class_counts[object_id][0] > class_counts[object_id][1]:
            filtered_objects[object_id] = count

    return filtered_objects


In [ ]:
PATH_PREDICTIONS = f"../data/predictions/{timestamp}"

for sequence in SEQUENCES:
    print("-"*50)
    print(sequence)

    for case in ["baseline", timestamp]:
        object_counts, class_counts = count_occurrences(f'../data/predictions/{case}/{sequence}.txt', case=case)
        filtered_objects = filter_counts(object_counts, class_counts)

        print(case)
        print(filtered_objects)